In [12]:
# playground for GNN data processing

import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch as th


import os

In [13]:
# extract filenames from npy_GNN folder

def get_filenames(directory):
    filenames = []
    for file in os.listdir(f"../npy_GNN/{directory}"):
        if file[-4:] == ".npy":
            filenames.append(file)
    filenames.sort()
    return filenames

names = get_filenames("2024-05-15_11:57:51_dynamic")
print(names)

# create a second list with the same filenames but without everything after the first underscore

def get_filenames_no_time(directory):
    filenames = []
    for file in os.listdir(f"../npy_GNN/{directory}"):
        if file[-4:] == ".npy":
            filenames.append(file)
    filenames.sort()
    for i in range(len(filenames)):
        filenames[i] = filenames[i].split("_")[0] + ".npy"
    return filenames

names_no_time = get_filenames_no_time("2024-05-15_11:57:51_dynamic")
types = len(set(names_no_time))

samples = len(names) // types
print(types, samples)

['CoarseResPoints_10.789_x_-0.283_y_0.959.npy', 'CoarseResPoints_11.324_x_-0.578_y_0.816.npy', 'CoarseResPoints_11.577_x_-0.049_y_0.999.npy', 'CoarseResPoints_12.313_x_0.79_y_-0.614.npy', 'CoarseResPoints_13.037_x_0.269_y_0.963.npy', 'CoarseResPoints_13.537_x_0.798_y_0.603.npy', 'CoarseResPoints_13.746_x_-0.944_y_0.33.npy', 'CoarseResPoints_14.492_x_1.0_y_0.026.npy', 'CoarseResPoints_15.447_x_-0.979_y_0.202.npy', 'CoarseResPoints_15.796_x_0.671_y_-0.741.npy', 'CoarseResPoints_15.811_x_0.434_y_0.901.npy', 'CoarseResPoints_16.065_x_-0.327_y_-0.945.npy', 'CoarseResPoints_16.365_x_-0.777_y_0.629.npy', 'CoarseResPoints_16.95_x_-0.085_y_-0.996.npy', 'CoarseResPoints_17.222_x_-0.673_y_0.74.npy', 'CoarseResPoints_17.284_x_-0.365_y_-0.931.npy', 'CoarseResPoints_17.359_x_-0.877_y_0.48.npy', 'CoarseResPoints_17.643_x_1.0_y_0.005.npy', 'CoarseResPoints_19.718_x_0.161_y_0.987.npy', 'CoarseResPoints_20.173_x_-0.817_y_0.577.npy', 'CoarseResPoints_21.045_x_0.53_y_-0.848.npy', 'CoarseResPoints_22.494_x

In [14]:
# create a graph 
#load displacement data 
high_res_displacement = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*3]}")
low_res_displacement = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[0]}")

high_res_velocity = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*4]}")
low_res_velocity = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*5]}")

#node features are just high res displacement and velocity horizontally stacked
node_features = np.hstack((high_res_displacement, high_res_velocity))
print(node_features.shape)
edge_index = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*2]}")[:, :2].T
edge_attr = np.load(f"../npy_GNN/2024-05-15_11:57:51_dynamic/{names[samples*2]}")[:, 2]

print(edge_index)
print(edge_attr.shape)

y = high_res_displacement - low_res_displacement

print(y.shape)

data = Data(x=th.tensor(node_features, dtype=th.float32), edge_index=th.tensor(edge_index, dtype=th.long), edge_attr=th.tensor(edge_attr, dtype=th.float32), y=th.tensor(y, dtype=th.float32))

# create a function to transform the data into a list of Data objects

def create_data_list(directory):
    names = get_filenames(directory)
    names_no_time = get_filenames_no_time(directory)
    types = len(set(names_no_time))
    samples = len(names) // types
    data_list = []
    for i in range(samples):
        high_res_displacement = np.load(f"../npy_GNN/{directory}/{names[samples*3+i]}")
        low_res_displacement = np.load(f"../npy_GNN/{directory}/{names[i]}")
        high_res_velocity = np.load(f"../npy_GNN/{directory}/{names[samples*4+i]}")
        node_features = np.hstack((high_res_displacement, high_res_velocity))
        edge_index = np.load(f"../npy_GNN/{directory}/{names[samples*2+i]}")[:, :2].T
        edge_attr = np.load(f"../npy_GNN/{directory}/{names[samples*2+i]}")[:, 2]
        y = high_res_displacement - low_res_displacement
        data = Data(x=th.tensor(node_features, dtype=th.float32), edge_index=th.tensor(edge_index, dtype=th.long), edge_attr=th.tensor(edge_attr, dtype=th.float32), y=th.tensor(y, dtype=th.float32))
        data_list.append(data)
    return data_list

data_list = create_data_list("2024-05-16_14:12:17_dynamic")
print(len(data_list))

(75, 6)
[[ 1. 16.  0. 16. 15.  2. 17.  1. 17.  3. 18.  2. 18.  4. 19.  3. 19.  5.
  20.  4. 20.  6. 21.  5. 21.  7. 22.  6. 22.  8. 23.  7. 23.  9. 24.  8.
  24. 10. 25.  9. 25. 11. 26. 10. 26. 12. 27. 11. 27. 13. 28. 12. 28. 14.
  29. 13. 29. 16. 31. 31. 30. 17. 32. 32. 18. 33. 33. 19. 34. 34. 20. 35.
  35. 21. 36. 36. 22. 37. 37. 23. 38. 38. 24. 39. 39. 25. 40. 40. 26. 41.
  41. 27. 42. 42. 28. 43. 43. 29. 44. 44. 31. 46. 46. 45. 32. 47. 47. 33.
  48. 48. 34. 49. 49. 35. 50. 50. 36. 51. 51. 37. 52. 52. 38. 53. 53. 39.
  54. 54. 40. 55. 55. 41. 56. 56. 42. 57. 57. 43. 58. 58. 44. 59. 59. 46.
  61. 61. 60. 47. 62. 62. 48. 63. 63. 49. 64. 64. 50. 65. 65. 51. 66. 66.
  52. 67. 67. 53. 68. 68. 54. 69. 69. 55. 70. 70. 56. 71. 71. 57. 72. 72.
  58. 73. 73. 59. 74. 74.]
 [16.  0.  1. 15.  0. 17.  1.  2. 16. 18.  2.  3. 17. 19.  3.  4. 18. 20.
   4.  5. 19. 21.  5.  6. 20. 22.  6.  7. 21. 23.  7.  8. 22. 24.  8.  9.
  23. 25.  9. 10. 24. 26. 10. 11. 25. 27. 11. 12. 26. 28. 12. 13. 27. 29.
  1

In [15]:
# build a GNN model that takes as input a Data object and outputs a prediction of shape (250, 3)
from torch.functional import F

class Net(th.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(6, 16)
        self.conv_mid = GCNConv(16, 16)
        self.conv2 = GCNConv(16, 3)
        self.repeat = 10 

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        for i in range(self.repeat):
            x = self.conv_mid(x, edge_index, edge_attr)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return x

model = Net()

# train the model

from torch_geometric.loader import DataLoader

device = th.device('cuda' if th.cuda.is_available() else 'cpu')
model = model.to(device)
data_list = [data.to(device) for data in data_list]


loader = DataLoader(data_list, batch_size=32, shuffle=True)

model.train()

optimizer = th.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(2000):
    for data in loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out, data.y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss {loss.item()}")
  



Epoch 0, Loss 0.23765960335731506
Epoch 1, Loss 0.2894006669521332
Epoch 2, Loss 0.13791224360466003
Epoch 3, Loss 0.2344426065683365
Epoch 4, Loss 0.3105621337890625
Epoch 5, Loss 0.08280014991760254
Epoch 6, Loss 0.07269585132598877
Epoch 7, Loss 0.5815435647964478
Epoch 8, Loss 0.5006441473960876
Epoch 9, Loss 0.5317392945289612
Epoch 10, Loss 0.742400050163269
Epoch 11, Loss 0.15733395516872406
Epoch 12, Loss 0.3578960597515106
Epoch 13, Loss 0.2383035272359848
Epoch 14, Loss 0.11961913853883743
Epoch 15, Loss 0.22317026555538177
Epoch 16, Loss 0.276107519865036
Epoch 17, Loss 0.5254722237586975
Epoch 18, Loss 0.19623059034347534
Epoch 19, Loss 0.21474575996398926
Epoch 20, Loss 0.25050511956214905
Epoch 21, Loss 0.17876353859901428
Epoch 22, Loss 0.1338621824979782
Epoch 23, Loss 0.3019762635231018
Epoch 24, Loss 0.09460865706205368
Epoch 25, Loss 0.6272963881492615
Epoch 26, Loss 0.21866735816001892
Epoch 27, Loss 0.4197438657283783
Epoch 28, Loss 0.2739923894405365
Epoch 29, Los

In [16]:
# test the model

model.eval()

for data in loader:
    out = model(data)
    print(out.shape)
    print(data.y.shape)
    print(F.mse_loss(out, data.y).item())


torch.Size([2400, 3])
torch.Size([2400, 3])
0.10853355377912521
torch.Size([2400, 3])
torch.Size([2400, 3])
0.16623879969120026
torch.Size([2400, 3])
torch.Size([2400, 3])
0.20442864298820496
torch.Size([2400, 3])
torch.Size([2400, 3])
0.17800293862819672
torch.Size([2400, 3])
torch.Size([2400, 3])
0.1783241182565689
torch.Size([2400, 3])
torch.Size([2400, 3])
0.11298210173845291
torch.Size([2400, 3])
torch.Size([2400, 3])
0.26745638251304626
torch.Size([2400, 3])
torch.Size([2400, 3])
0.17699655890464783
torch.Size([2400, 3])
torch.Size([2400, 3])
0.3378833532333374
torch.Size([2400, 3])
torch.Size([2400, 3])
0.18071849644184113
torch.Size([2400, 3])
torch.Size([2400, 3])
0.20018310844898224
torch.Size([2400, 3])
torch.Size([2400, 3])
0.11308273673057556
torch.Size([2400, 3])
torch.Size([2400, 3])
0.23241764307022095
torch.Size([2400, 3])
torch.Size([2400, 3])
0.15112733840942383
torch.Size([2400, 3])
torch.Size([2400, 3])
0.25183865427970886
torch.Size([2400, 3])
torch.Size([2400, 3]